In [1]:
#EDA

In [2]:
import torch

In [ ]:
#pip install torch==2.0.0
#pip install pysentimiento==0.7.2
#pip install evaluate==0.4.0
#pip install datasets==2.14.5

In [ ]:
#!pip install pysentimiento transformers datasets accelerate evaluate

In [4]:
import datasets
import evaluate

2024-03-17 13:43:39.955768: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-17 13:43:39.990248: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-17 13:43:39.990274: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-17 13:43:39.991208: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-17 13:43:39.996803: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructio

In [5]:
import numpy as np
from datasets import load_dataset

In [6]:
import ipywidgets as widgets

In [7]:
#Helper Functions

In [8]:
# label to name
def label2name(x):
    if x == 0:
        return "Negative"
    if x == 1:
        return "Neutral"
    if x == 2:
        return "Positive"

In [9]:

f1_metric = evaluate.load("f1")
recall_metric = evaluate.load("recall")

def compute_metrics (eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis = -1)
    
    results = {}
    results.update(f1_metric.compute(predictions=preds, references = labels, average="macro"))
    results.update(recall_metric.compute(predictions=preds, references = labels, average="macro"))
    return results

In [10]:
#Read Data


In [11]:
data_files = {"train": "data/train.csv", "validation": "data/val.csv", "test": "data/test.csv"}
ds = load_dataset("csv", data_files=data_files)

In [12]:
ds

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_name', 'tokenized_text', 'sent_token_length', 'sent_bert_token_length', 'char_count', 'Character Count'],
        num_rows: 9133
    })
    validation: Dataset({
        features: ['text', 'label', 'label_name', 'tokenized_text', 'sent_token_length', 'sent_bert_token_length', 'char_count', 'Character Count'],
        num_rows: 1128
    })
    test: Dataset({
        features: ['text', 'label', 'label_name', 'tokenized_text', 'sent_token_length', 'sent_bert_token_length', 'char_count', 'Character Count'],
        num_rows: 1015
    })
})

In [13]:
#push to hugging face

In [41]:
from huggingface_hub import notebook_login

In [ ]:
#https://huggingface.co/settings/tokens

In [44]:
notebook_login()

In [45]:
ds.push_to_hub("jairleo95/social-media-peruvian-sentiment")

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

In [15]:
ds["train"].features

{'text': Value(dtype='string', id=None),
 'label': Value(dtype='int64', id=None),
 'label_name': Value(dtype='string', id=None),
 'tokenized_text': Value(dtype='string', id=None),
 'sent_token_length': Value(dtype='int64', id=None),
 'sent_bert_token_length': Value(dtype='int64', id=None),
 'char_count': Value(dtype='int64', id=None),
 'Character Count': Value(dtype='int64', id=None)}

In [16]:
ds["test"]["label"][:10]

[1, 0, 0, 0, 2, 1, 2, 0, 0, 1]

In [17]:
df = ds['train'].to_pandas()
df.head()

,text,label,label_name,tokenized_text,sent_token_length,sent_bert_token_length,char_count,Character Count
0,"Nada como estar en lugares espectaculares, Man...",2,Positive,Nada como estar en lugares espectaculares Man...,18,24,118,118
1,La policía anda pensando que eres un subversivo,1,Neutral,La policía anda pensando que eres un subversivo,8,10,47,47
2,MAYRA GOÑY ...DEJALOS A TODOS EN PINDINGA!!! I...,0,Negative,MAYRA GOÑY DEJALOS A TODOS EN PINDINGA I...,10,19,62,62
3,Vamos a poner a un huevón más y nos vamos,0,Negative,Vamos a poner a un huevón más y nos vamos,10,11,41,41
4,Extraño las cachemas con yucas 😔,1,Neutral,Extraño las cachemas con yucas 😔,6,8,32,32


In [18]:
# encode label and mapping label name
#df["label"] = df["label"].apply(lambda x: label_encode(x))
df["label_name"] = df["label"].apply(lambda x: label2name(x))

In [19]:
# clean text, lowercase and remove punk
#df["text"] = df["text"].apply(lambda x: remove_punct(clean(remove_emoji(x).lower())[0][0]))

In [20]:
df.head()

,text,label,label_name,tokenized_text,sent_token_length,sent_bert_token_length,char_count,Character Count
0,"Nada como estar en lugares espectaculares, Man...",2,Positive,Nada como estar en lugares espectaculares Man...,18,24,118,118
1,La policía anda pensando que eres un subversivo,1,Neutral,La policía anda pensando que eres un subversivo,8,10,47,47
2,MAYRA GOÑY ...DEJALOS A TODOS EN PINDINGA!!! I...,0,Negative,MAYRA GOÑY DEJALOS A TODOS EN PINDINGA I...,10,19,62,62
3,Vamos a poner a un huevón más y nos vamos,0,Negative,Vamos a poner a un huevón más y nos vamos,10,11,41,41
4,Extraño las cachemas con yucas 😔,1,Neutral,Extraño las cachemas con yucas 😔,6,8,32,32


In [21]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "pysentimiento/robertuito-base-uncased"

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=3
)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.model_max_length = 128

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at pysentimiento/robertuito-base-uncased and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
# columns = ['input_ids', 'attention_mask', 'label']
# ds.set_format(type='torch', columns=columns)
# ds

In [23]:
from pysentimiento.preprocessing import preprocess_tweet
preprocessed_ds = ds.map(lambda ex: {"text": preprocess_tweet(ex["text"], lang="es")})

In [24]:
tokenized_ds = preprocessed_ds.map(
    lambda batch: tokenizer(
        batch["text"], padding=True, truncation=True
        ),
    batched=True, batch_size=32
)

In [25]:
tokenized_ds

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_name', 'tokenized_text', 'sent_token_length', 'sent_bert_token_length', 'char_count', 'Character Count', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9133
    })
    validation: Dataset({
        features: ['text', 'label', 'label_name', 'tokenized_text', 'sent_token_length', 'sent_bert_token_length', 'char_count', 'Character Count', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1128
    })
    test: Dataset({
        features: ['text', 'label', 'label_name', 'tokenized_text', 'sent_token_length', 'sent_bert_token_length', 'char_count', 'Character Count', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1015
    })
})

In [26]:
tokenized_ds['train']['input_ids'][0][::10]

[0, 863, 4143, 1326, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [27]:
tokenized_ds['train']['attention_mask'][0][::10]

[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [28]:
tokenized_ds['train']['text'][0]

'Nada como estar en lugares espectaculares, Manolito no me pierdo tus programas asi que pilas para nuevas aventurillas emoji cara feliz con ojos sonrientes emoji'

In [29]:
#!pip install ipdb

In [30]:
#import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
model.cuda()

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=1)
      (position_embeddings): Embedding(130, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [31]:
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding

training_args = TrainingArguments(
    per_device_train_batch_size=32,
    output_dir="trainer_robertuito",
    warmup_ratio=0.1,
    learning_rate=5e-5,
    do_eval=True,
    evaluation_strategy="epoch",
    num_train_epochs=5,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["validation"],
    compute_metrics=compute_metrics,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
)

In [32]:
trainer.train()

{'eval_loss': 0.6280988454818726, 'eval_f1': 0.7011731673470102, 'eval_recall': 0.7042052938005089, 'eval_runtime': 4.8757, 'eval_samples_per_second': 231.349, 'eval_steps_per_second': 28.919, 'epoch': 1.0}


KeyboardInterrupt: 

In [ ]:
#Test on validation set

In [ ]:
trainer.evaluate(tokenized_ds["test"])

In [ ]:
#model.load_state_dict(torch.load(f'./_BERT_epoch_3.model', map_location=torch.device('cpu')))

In [ ]:
#Error Analisys

In [ ]:
import pandas as pd

In [ ]:
val_df = ds['validation'].to_pandas()
#val_df

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
# step by step predictions on dataframe
# We do this to view predictions in the pandas dataframe and easily filter them and perform error analysis.
pred_final = []

for i, row in tqdm(val_df.iterrows(), total=val_df.shape[0]):
    predictions = []

    text = row["text"]
    encoded_data_test_single = tokenizer.batch_encode_plus([text], 
    # add_special_tokens=config.add_special_tokens, 
    # return_attention_mask=config.return_attention_mask, 
    # pad_to_max_length=config.pad_to_max_length, 
    max_length=128,
    # return_tensors=config.return_tensors
    return_tensors="pt"
    )
    input_ids_test = encoded_data_test_single['input_ids']
    attention_masks_test = encoded_data_test_single['attention_mask']

    
    inputs = {'input_ids':      input_ids_test.to(device),
              'attention_mask':attention_masks_test.to(device),
             }

    with torch.no_grad():        
        outputs = model(**inputs)
    
    logits = outputs[0]
    logits = logits.detach().cpu().numpy()
    predictions.append(logits)
    predictions = np.concatenate(predictions, axis=0)
    pred_final.append(np.argmax(predictions, axis=1).flatten()[0])

In [ ]:
# add pred into val_df
val_df["pred"] = pred_final

In [ ]:
#  Add control column for easier wrong and right predictions
control = val_df.pred.values == val_df.label.values
val_df["control"] = control

In [ ]:
# filtering false predictions
val_df = val_df[val_df.control == False]

In [ ]:
# label to intent mapping
name2label = {"Negative":0,
              "Neutral":1,
             "Positive":2
             }
label2name = {v: k for k, v in name2label.items()}

val_df["pred_name"] = val_df.pred.apply(lambda x: label2name.get(x)) 

In [ ]:
from sklearn.metrics import confusion_matrix

# We create a confusion matrix to better observe the classes that the model confuses.
pred_name_values = val_df.pred_name.values
label_values = val_df.label_name.values
confmat = confusion_matrix(label_values, pred_name_values, labels=list(name2label.keys()))

In [ ]:
confmat

In [ ]:
df_confusion_val = pd.crosstab(label_values, pred_name_values)
df_confusion_val

In [ ]:
# save confissuan matrix df
df_confusion_val.to_csv("val_df_confusion.csv")

## Inference

In [ ]:
encoded_data_test = tokenizer.batch_encode_plus(
    val_df.text.values, 
    add_special_tokens=config.add_special_tokens, 
    return_attention_mask=config.return_attention_mask, 
    pad_to_max_length=config.pad_to_max_length,
    max_length=128, 
    return_tensors=config.return_tensors
)

In [ ]:
#Evaluate the Model Qualitatively (Human Evaluation)

In [ ]:
#Evaluate the Model Quantitatively (with F1 Metric)

In [ ]:
texts = val_df[0:20]['text']
human_baseline_labels = val_df[0:20]['label']

In [ ]:
texts

In [ ]:
human_baseline_labels

In [ ]:
#original_model_summaries = []
model_classifications = []

In [ ]:
for _, text in enumerate(texts):
    input_ids = tokenizer(text, return_tensors="pt", max_length=128).input_ids.to('cuda')
    logits = model(input_ids).logits
    probabilities = logits.softmax(dim=-1).tolist()[0]
    model_classifications.append(np.argmax(probabilities).flatten()[0])

In [ ]:
model_classifications